## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cherskiy,RU,68.75,161.30,46.00,73,100,6.29,overcast clouds
1,1,Beringovskiy,RU,63.05,179.32,47.34,83,100,22.95,overcast clouds
2,2,Qaanaaq,GL,77.48,-69.36,30.88,83,0,5.30,clear sky
3,3,Avarua,CK,-21.21,-159.78,77.00,69,44,18.34,scattered clouds
4,4,Katsuura,JP,35.13,140.30,95.00,62,75,4.70,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Avarua,CK,-21.21,-159.78,77.00,69,44,18.34,scattered clouds
5,5,Carnarvon,AU,-24.87,113.63,75.20,29,0,12.75,clear sky
14,14,Portel,BR,-1.94,-50.82,76.53,80,47,3.18,scattered clouds
18,18,San Pedro De Macoris,DO,18.46,-69.30,80.60,83,20,5.82,few clouds
24,24,Hamilton,US,39.18,-84.53,77.00,61,40,5.82,scattered clouds
26,26,Hobyo,SO,5.35,48.53,76.42,87,99,21.54,overcast clouds
47,47,Tateyama,JP,34.98,139.87,89.60,62,75,4.70,broken clouds
49,49,Lexington Park,US,38.27,-76.45,77.00,83,1,6.93,thunderstorm
51,51,Atuona,PF,-9.80,-139.03,79.18,77,13,20.24,few clouds
52,52,Victoria,HK,22.29,114.16,89.01,66,20,4.00,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# There are no empty rows because there are values in all the cells
preferred_cities_df.notnull().sum()

City_ID                235
City                   235
Country                235
Lat                    235
Lng                    235
Max Temp               235
Humidity               235
Cloudiness             235
Wind Speed             235
Current Description    235
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,77.00,scattered clouds,-21.21,-159.78,
5,Carnarvon,AU,75.20,clear sky,-24.87,113.63,
14,Portel,BR,76.53,scattered clouds,-1.94,-50.82,
18,San Pedro De Macoris,DO,80.60,few clouds,18.46,-69.30,
24,Hamilton,US,77.00,scattered clouds,39.18,-84.53,
26,Hobyo,SO,76.42,overcast clouds,5.35,48.53,
47,Tateyama,JP,89.60,broken clouds,34.98,139.87,
49,Lexington Park,US,77.00,thunderstorm,38.27,-76.45,
51,Atuona,PF,79.18,few clouds,-9.80,-139.03,
52,Victoria,HK,89.01,light rain,22.29,114.16,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# Check contents of hotel dataframe - there 235 records.
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,77.00,scattered clouds,-21.21,-159.78,Paradise Inn
5,Carnarvon,AU,75.20,clear sky,-24.87,113.63,Hospitality Carnarvon
14,Portel,BR,76.53,scattered clouds,-1.94,-50.82,Pousada Avenida
18,San Pedro De Macoris,DO,80.60,few clouds,18.46,-69.30,Apartahotel Guaraguao
24,Hamilton,US,77.00,scattered clouds,39.18,-84.53,Six Acres Bed & Breakfast
...,...,...,...,...,...,...,...
655,Guiren,CN,75.31,overcast clouds,33.67,118.19,Suyuan Hotel
661,Puerto Cabezas,NI,79.83,few clouds,14.04,-83.39,Hotel Caribe
668,Iberia,US,80.60,clear sky,29.95,-91.75,back to back rv park
669,Arlit,NE,87.96,few clouds,18.74,7.39,Hôtel Telwa Bungalow


In [11]:
# 7. Drop the rows where there is no Hotel Name - there are now 226 complete records.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,77.00,scattered clouds,-21.21,-159.78,Paradise Inn
5,Carnarvon,AU,75.20,clear sky,-24.87,113.63,Hospitality Carnarvon
14,Portel,BR,76.53,scattered clouds,-1.94,-50.82,Pousada Avenida
18,San Pedro De Macoris,DO,80.60,few clouds,18.46,-69.30,Apartahotel Guaraguao
24,Hamilton,US,77.00,scattered clouds,39.18,-84.53,Six Acres Bed & Breakfast
...,...,...,...,...,...,...,...
655,Guiren,CN,75.31,overcast clouds,33.67,118.19,Suyuan Hotel
661,Puerto Cabezas,NI,79.83,few clouds,14.04,-83.39,Hotel Caribe
668,Iberia,US,80.60,clear sky,29.95,-91.75,back to back rv park
669,Arlit,NE,87.96,few clouds,18.74,7.39,Hôtel Telwa Bungalow


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))